In [6]:
import import_ipynb
import random
import sensors
from environment import Environment

In [5]:
# As per slide 52, it receives the environment, uses simple rules (here random) to chose action (here it is moving to a position)
def simple_reflex_agent(environ):
    # First it checks which positions dirty
    dirty = []
    for i in range(len(locations)):
        for j in range(len(locations[i])):
            if locations[i][j]:
                dirty.append([i,j])
    
    loc = random.choice(dirty)
    return [ loc[0] - 2, loc[1] - 2 ]

class VacuumCleaner:
    def __init__(self):
        self.environ = Environment()
        self.position_sensor = sensors.PositionSensor()
        self.visual_sensor = sensors.VisualSensor()
    
    def attempt_clean(self):
        print("Cleaning some location")
        # will always return 5*5 matrix
        locations = self.visual_sensor.scan_dirty_regions(self)

        # We pass a limited envionment to Reflex agent
        move = simple_reflex_agent(locations)

        # not logically correct for sensor to do move i think, but for now it will work
        self.position_sensor.move(move)


In [4]:
VacuumCleaner

__main__.VacuumCleaner